03/02/2024
This code has been copied from 2 notebooks:

231215_dataset_building -> parts of resizing

240131_nnproject_img_txt_topq -> the working parts, images written as bytes to parquet file, together with the description

In [ ]:
import os, glob
import pandas as pd
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageOps, UnidentifiedImageError
import pyarrow.parquet as pq
import pyarrow as pa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path for input folder
# data_folder = '/content/drive/MyDrive/cogsci - sem3/Advanced_applications_of_neural_networks/projekt/huggin-face/data'
# 02/01/24
data_folder = '/content/drive/MyDrive/cogsci - sem3/Advanced_applications_of_neural_networks/projekt/temporal'

note that I changed the structure of the folders.

In [ ]:
output_folder = '/content/drive/MyDrive/cogsci - sem3/Advanced_applications_of_neural_networks/projekt/huggin-face/eco-archi/resized'

# 2 Process images - cropping to a given ratio and resizing

In [ ]:
# Resize image to a ratio 16:9
# if needed add white padding around the image
def resize_16_9(file_path):
    try:
        # Image resolution
        target_width = 255
        target_height = 143

        # Open image
        with Image.open(file_path) as img:

            # Current dimensions of the image
            width, height = img.size

            # Calculate the target height based on the 16:9 ratio
            target_height_calculated = int((16 / 9) * width)

            # Calculate the padding needed to achieve the 16:9 ratio
            padding_top = max(0, (target_height - target_height_calculated) // 2)
            padding_bottom = max(0, target_height - target_height_calculated - padding_top)

            # Add white padding around the image
            padded_img = ImageOps.pad(img, (target_width, target_height), method=0, color=(255, 255, 255))

            # Save the processed image
            output_path = os.path.join(output_folder, os.path.basename(file_path))
            padded_img.save(output_path)
            print(f"Saved")

    except (UnidentifiedImageError, IOError) as e:
        print(f"Error processing {file_path}: {e}")

In [ ]:
# Process each image file
def process_images_in_folder(folder_path, output_path, method):

    global output_folder
    output_folder = output_path
    os.makedirs(output_folder, exist_ok=True)

    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Process each image file
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        method(file_path)

In [ ]:
process_images_in_folder(data_folder, output_folder, resize_16_9)

Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved
Saved


#3 Creating a dataset

The script is changed comparing to 15/12/24 because it contains code for parquet file with images described as bytes, instead of 3 rgb channels, and for each images there is a place for description.
02/02/24 This script I used for 37 images I have description for.


In [ ]:
import pyarrow as pa
import numpy as np
import PIL

In [ ]:
data_folder = '/content/drive/MyDrive/cogsci - sem3/Advanced_applications_of_neural_networks/projekt/huggin-face/eco-archi/resized'

In [ ]:
metadata = pd.read_csv('/content/drive/MyDrive/cogsci - sem3/Advanced_applications_of_neural_networks/projekt/huggin-face/metadata.csv')

In [ ]:
df = metadata

In [ ]:
df[1:10]

,file_name,text,folder_name
1,nordlicht-house-face-buro-florian-schatz_4.jpg,This is a detached house in a snowy landscape....,batch-1
2,oficinas-polaris-north-b720-fermin-vazquez-arq...,This is a 5 floor office building in a pitched...,batch-1
3,pravaah-workplace-the-grid-architects_4.jpg,"This a small office building, which has a very...",batch-1
4,The_First_ECO-SANDWICH_House_(10).jpg,House is a unique example of a holistic and in...,batch-1
5,zen-spaces-residence-sanjay-puri-architects_1.jpg,The house is a composition of cuboid volumes a...,batch-1
6,04-marc-koehler-vtc-edegem-c-dennis-de-smet.jpg,It is a regional center for culture and leisur...,batch-1
7,biblioteca-de-sant-marti-sarroca-valor-llimos_...,The library building is located inside the par...,batch-1
8,casa-ene-estudio-albar_1.jpg,"A house with a cork façade, wooden windows fac...",batch-1
9,dsc0588.jpg,"House is places in a rocky plot, which has bee...",batch-1


In [ ]:
df[df["file_name"]=="gurdau-winery-ales-fiala_5.jpg"]

,file_name,text,folder_name
11,gurdau-winery-ales-fiala_5.jpg,This is a wine house integrated into the lands...,batch-1


In [ ]:
records = []

In [ ]:
for img_name in df['file_name']:
            img_path = os.path.join(data_folder, img_name)
            # Check if the file exists
            if os.path.isfile(img_path):
                text_data = str(df.loc[df['file_name'] == img_name, 'text'].values[0])  # Convert to string
                with PIL.Image.open(img_path) as im:
                  records.append({
                    "image": im.tobytes(),
                    "text": text_data
                    })
            else:
                print(f"Warning: File not found - {img_path}")

In [ ]:
# Convert the list of records to a Pandas DataFrame
df_records = pd.DataFrame(records)

In [ ]:
# just in case there were any
# df_records.dropna(inplace=True)

In [ ]:
# check if there are any missing data
len(df) == len(df_records)

True

In [ ]:
df_records

In [ ]:
table = pa.Table.from_pylist(records)

In [ ]:
parquet_name = "data_bytes_240302"

In [ ]:
parquet_file_path = f"/content/drive/MyDrive/cogsci - sem3/Advanced_applications_of_neural_networks/projekt/huggin-face/arch-facade2s/train/{parquet_name}.parquet"
pa.parquet.write_table(table, parquet_file_path)

now I am checking again if the image is showing properly

In [ ]:
table4 = pq.read_table(f'/content/drive/MyDrive/cogsci - sem3/Advanced_applications_of_neural_networks/projekt/huggin-face/arch-facade2s/train/{parquet_name}.parquet')

In [ ]:
df_4 = table4.to_pandas()

In [ ]:
# Get the 'image' column from the PyArrow Table
image_column = table['image']

# Convert the Arrow Array to a NumPy array
image_np = np.frombuffer(image_column[0].as_py(), dtype=np.uint8)

# Replace these placeholders with the actual dimensions of your original images
original_height = 143
original_width = 255
original_channels = 3

original_shape = (original_height, original_width, original_channels)

# Reshape the flattened array back to the original shape
reshaped_image = np.reshape(image_np, original_shape)

# Convert NumPy array to PIL Image
image = PIL.Image.fromarray(reshaped_image)

# Display the image
image.show()

In [ ]:
# Save the image to a temporary file
image.save('temporary_image2.png')

# Open the saved image using a system default image viewer
import subprocess
subprocess.run(['open', 'temporary_image2.png'])

CompletedProcess(args=['open', 'temporary_image2.png'], returncode=3)

now I need to solve the problems with
1. so in the folder resized, several of my images that I have text (caption) for are missing
2. and I am thinking that perhaps I should upload all my images I have a text for, into one folder
3. again copy the code for resizing
4. and thereafter take the above code to run it to finally have a proper *.parquet file
5. which I can upload in my dataset in hugginface
6. after that I can try to implement this:
https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/image_captioning_blip.ipynb#scrollTo=93od71o_qq_V
7. however, I am not sure if this kind of representation is suitable for the next steps, which is dreambooth and

yeah it turned out about right, it is impossible to read the data

In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


In [ ]:
from datasets import Features, Image, Value

features = Features({"image": Image(), "text": Value("string")})
schema = features.arrow_schema
print(schema.metadata)
# {b'huggingface': b'{"info": {"features": {"image": {"_type": "Image"}, "text": {"dtype": "string", "_type": "Value"}}}}'}

{b'huggingface': b'{"info": {"features": {"image": {"_type": "Image"}, "text": {"dtype": "string", "_type": "Value"}}}}'}


In [ ]:
from datasets import load_dataset

In [ ]:
dataset_builder

In [ ]:
load_dataset

In [ ]:
PIL.JpegImagePlugin.JpegImageFile

PIL.JpegImagePlugin.JpegImageFile

230203 It turned out it's not necessary to have *.parquet file uploaded in huggingface, nevertheless in the future I want to have this type of file. Uploading metadata.csv to image file does the work.